In [4]:
import mediapipe as mp
import cv2
import numpy as np

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Basic Detection

In [6]:
# Getting video feed
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        # Rendering
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.564463
y: 0.714277
z: -0.957843
visibility: 0.998558
, x: 0.60514975
y: 0.6605176
z: -0.87748605
visibility: 0.9970667
, x: 0.62459904
y: 0.6699103
z: -0.8784714
visibility: 0.9968201
, x: 0.64465725
y: 0.6782183
z: -0.8782455
visibility: 0.9970914
, x: 0.5427309
y: 0.64655113
z: -0.87234485
visibility: 0.9985141
, x: 0.5265714
y: 0.6463387
z: -0.87334174
visibility: 0.998816
, x: 0.5085094
y: 0.6485996
z: -0.8740948
visibility: 0.99883
, x: 0.67630756
y: 0.71499276
z: -0.41790205
visibility: 0.9981078
, x: 0.48572356
y: 0.68495786
z: -0.40378484
visibility: 0.99834
, x: 0.60007995
y: 0.7972702
z: -0.7797225
visibility: 0.99482673
, x: 0.5211404
y: 0.7809956
z: -0.7835838
visibility: 0.9956975
, x: 0.7886878
y: 1.0380981
z: -0.16797572
visibility: 0.9313916
, x: 0.35964644
y: 0.9933826
z: -0.18559729
visibility: 0.94160223
, x: 0.79670197
y: 1.339978
z: -0.5151728
visibility: 0.15859365
, x: 0.27241203
y: 1.3679063
z: -0.58647263
visibility: 0.059307616
, x: 0.7773715
y: 1.090346

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7816762
y: 1.0222372
z: -0.120674506
visibility: 0.9627398

In [8]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.824509
y: 1.3290795
z: -0.24261627
visibility: 0.24421911

# 3. Calculate Angles

In [9]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [10]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]


In [11]:
shoulder, elbow, wrist

([0.781676173210144, 1.0222371816635132],
 [0.8245090246200562, 1.329079508781433],
 [0.7982765436172485, 1.169919490814209])

In [12]:
calculate_angle(shoulder, elbow, wrist)

1.4125458796157557

In [13]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # or any height you prefer
stage = ""
counter = 0
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, 
                  min_tracking_confidence=0.5,
                  model_complexity=2,
                  static_image_mode=False,
                  smooth_landmarks=False,
                          ) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        frame = cv2.flip(frame, 1)
        
        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for i, landmark in enumerate(landmarks):
                print(f"Landmark {i}: x={landmark.x}, y={landmark.y}, z={landmark.z}")

            # Get coordinates
            angles_to_calculate = {
                "right_elbow_right_shoulder_right_hip": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                ],
                "left_elbow_left_shoulder_left_hip": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                ],
                "right_knee_mid_hip_left_knee": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
                     (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                ],
                "right_hip_right_knee_right_ankle": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
                ],
                "left_hip_left_knee_left_ankle": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
                ],
                "right_wrist_right_elbow_right_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                ],
                "left_wrist_left_elbow_left_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                ],
            }

            # Calculate and visualize angles
            angles = []
            for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
                angle = calculate_angle(points[0], points[1], points[2])
                angles.append(angle)
                cv2.putText(image, f'{angle_name}: {int(angle)}', 
                            (50, 100 + i * 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            if angles[3] > 160 and angles[4] > 160:
                stage = "up"
            if angles[3] < 90 and angles[4] < 90 and stage == "up":
                stage = "down"
                counter += 1
        else:
            print("No landmarks detected in this frame.")

            
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        # cv2.imshow('Mediapipe Feed', cv2.flip(image,1))
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Landmark 0: x=0.5490652322769165, y=0.729341447353363, z=-0.5701486468315125
Landmark 1: x=0.5740737915039062, y=0.673386812210083, z=-0.5212759971618652
Landmark 2: x=0.5893892645835876, y=0.6786558628082275, z=-0.5217692255973816
Landmark 3: x=0.6029014587402344, y=0.6842687129974365, z=-0.5216746926307678
Landmark 4: x=0.5302130579948425, y=0.6665328741073608, z=-0.5190128684043884
Landmark 5: x=0.5160130262374878, y=0.6677722930908203, z=-0.5197892785072327
Landmark 6: x=0.5054926872253418, y=0.6692529916763306, z=-0.5202686786651611
Landmark 7: x=0.6224444508552551, y=0.7193560004234314, z=-0.2331998646259308
Landmark 8: x=0.49154701828956604, y=0.700866162776947, z=-0.23070664703845978
Landmark 9: x=0.5755370259284973, y=0.805086076259613, z=-0.45619410276412964
Landmark 10: x=0.5186356902122498, y=0.7944146990776062, z=-0.46123212575912476
Landmark 11: x=0.6919963359832764, y=1.0187344551086426, z=-0.06764519214630127
Landmark 12: x=0.39590567350387573, y=0.9884572625160217, z=-

In [14]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import torch
# import nbimporter  # Import nbimporter

# from data import NeuralNetwork  
#  # Import your model class

# model = NeuralNetwork()  # Initialize the model
# model.load_state_dict(torch.load(r'D:\PROGRAMMING\BE proj24\data.pth'))  # Load the state dict
# model.eval() 

# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Get video feed
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) 
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Make detection
#         results = pose.process(image)

#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract landmarks
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark

#             # Prepare input data for your model
#             input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

#             # Predict next movement
#             with torch.no_grad():
#                 predicted_angles = model(input_tensor)
#                 predicted_angles = predicted_angles.squeeze().tolist()

#             # Display predicted movement
#             for i, angle in enumerate(predicted_angles):
#                 cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', 
#                             (50, 300 + i * 30),  # Adjust y-position to fit all angles
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA) 


#         # Render pose landmarks
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [15]:
# import sys
# sys.path.append(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.ipynb')
# import nbimporter   # Import nbimporter
# from data import NeuralNetwork

In [16]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import torch
# import nbimporter   # Import nbimporter
# from data import NeuralNetwork  # Import your model class
# # Initialize the model
# model = NeuralNetwork()
# model.load_state_dict(torch.load(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.pth'))
# model.eval()

# # Set up MediaPipe drawing and pose
# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Open video feed
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# stage = ""
# counter = 0

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2, smooth_landmarks=False) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Make pose detection
#         results = pose.process(image)

#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract landmarks and calculate angles
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark

#             # Define angle calculations (you can add other angles here)
#             angles_to_calculate = {
#                 "right_elbow_right_shoulder_right_hip": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                 ],
#                 "left_elbow_left_shoulder_left_hip": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                 ],
#                 "right_knee_mid_hip_left_knee": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
#                      (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                 ],
#                 "right_hip_right_knee_right_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
#                 ],
#                 "left_hip_left_knee_left_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
#                 ],
#                 "right_wrist_right_elbow_right_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                 ],
#                 "left_wrist_left_elbow_left_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                 ],
#             }

#             # Calculate angles and store them in the list for model input
#             angles = []
#             for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
#                 angle = calculate_angle(points[0], points[1], points[2])
#                 angles.append(angle)
#                 cv2.putText(image, f'{angle_name}: {int(angle)}', (50, 100 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

#             # Prepare input data for the model (use angles as input tensor)
#             input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

#             # Predict next movement
#             with torch.no_grad():
#                 predicted_angles = model(input_tensor).squeeze().tolist()

#             # Display predicted next movement angles
#             for i, angle in enumerate(predicted_angles):
#                 cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', (50, 300 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

#             # Rep counting logic
#             if angles[3] > 160 and angles[4] > 160:
#                 stage = "up"
#             if angles[3] < 90 and angles[4] < 90 and stage == "up":
#                 stage = "down"
#                 counter += 1

#         # Render rep counter and stage data
#         cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
#         cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

#         # Render pose landmarks
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#         # Display image
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()


In [17]:
# import os
# print(os.path.isfile(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.ipynb'))

# Holistic

In [18]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [19]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [20]:
# SPINE ANGLE

In [21]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Holistic

In [22]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [23]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [24]:
# SPINE ANGLE

In [25]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
